# Part 2 - Mapping Yelp Search Results

## Obective

- For this CodeAlong, we will be working with the Yelp API results from last class. 
- You will load in the .csv.gz of your yelp results and prepare the data for visualization.
- You will use Plotly Express to create an interactive map with all of the results.

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- [Advanced Transformations with Pandas - Part 1](https://login.codingdojo.com/m/376/12529/88086)
- [Advanced Transformations with Pandas - Part 2](https://login.codingdojo.com/m/376/12529/88088)

### Goal

- We want to create a map with every restaurant plotted as a scatter plot with detailed information that appears when we hover over a business
- We will use plotly express's `px.scatter_mapbox` function to accomplish this.
    - https://plotly.com/python/scattermapbox/
    
    - We will need a Mapbox API token for some of the options:
        - https://studio.mapbox.com/
    

# Loading Data from Part 1

In [1]:
## Plotly is not included in your dojo-env
!pip install plotly

In [2]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import json

## importing plotly 
import plotly.express as px

In [3]:
## Load in csv.gz
df = pd.read_csv('Data/Seattle-pizza.csv.gz')
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,EwrAoy7XbX8nfemZdDvupA,cornelly-seattle,Cornelly,https://s3-media4.fl.yelpcdn.com/bphoto/HlYYBT...,False,https://www.yelp.com/biz/cornelly-seattle?adju...,101,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 47.624577, 'longitude': -122.325577}","['delivery', 'pickup']","{'address1': '601 Summit Ave E', 'address2': '...",NaN,NaN,2578.701393,NaN
1,FVbwpNA1uZEGiM02N4XtUg,blotto-seattle,Blotto,https://s3-media3.fl.yelpcdn.com/bphoto/Q4Y1Cw...,False,https://www.yelp.com/biz/blotto-seattle?adjust...,40,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 47.61849, 'longitude': -122.31664}",[],"{'address1': '1830 12th Ave', 'address2': '', ...",1.206403e+10,(206) 403-1809,1686.862176,NaN
2,fxyHWmfzcdjImgQ_UYHoTw,bar-cotto-seattle,Bar Cotto,https://s3-media3.fl.yelpcdn.com/bphoto/07DoPF...,False,https://www.yelp.com/biz/bar-cotto-seattle?adj...,343,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"{'latitude': 47.6146934, 'longitude': -122.312...","['delivery', 'pickup']","{'address1': '1546 15th Ave', 'address2': '', ...",1.206839e+10,(206) 838-8081,1292.526796,$$
3,IrohtoYjnAR_vc6w6CRCxA,moto-seattle,Moto,https://s3-media2.fl.yelpcdn.com/bphoto/FBmvjy...,False,https://www.yelp.com/biz/moto-seattle?adjust_c...,143,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 47.56207, 'longitude': -122.38509}",['delivery'],"{'address1': '4526 42nd Ave SW', 'address2': '...",1.206421e+10,(206) 420-8880,8784.075722,$$
4,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media4.fl.yelpcdn.com/bphoto/A08GOZ...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,362,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 47.5976491915013, 'longitude': -1...",['delivery'],"{'address1': '525 Rainier Ave S', 'address2': ...",1.206330e+10,(206) 329-5133,2166.834011,$$


## Required Preprocessing 

- 1. We need to get the latitude and longitude for each business as separate columns.
- We also want to be able to show the restaurants:
    - name,
    - price range
    - address
    - and if they do delivery or takeout.

### Separating Latitude and Longitude

In [4]:
## use .apply pd.Series to convert a dict to columns
df['coordinates'].apply(pd.Series)

,0
0,"{'latitude': 47.624577, 'longitude': -122.325577}"
1,"{'latitude': 47.61849, 'longitude': -122.31664}"
2,"{'latitude': 47.6146934, 'longitude': -122.312..."
3,"{'latitude': 47.56207, 'longitude': -122.38509}"
4,"{'latitude': 47.5976491915013, 'longitude': -1..."
...,...
850,"{'latitude': 47.6157033283809, 'longitude': -1..."
851,"{'latitude': 47.6179959197781, 'longitude': -1..."
852,"{'latitude': 47.66410298, 'longitude': -122.31..."
853,"{'latitude': 47.61252119845868, 'longitude': -..."


- Why didn't that work???

In [6]:
## slice out a single test coordinate
test_coord = df.loc[1, 'coordinates']
test_coord

"{'latitude': 47.61849, 'longitude': -122.31664}"

- Its not a dictionary anymore!!! WTF??
    - CSV files cant store iterables (lists, dictionaries) so they get converted to strings.

### Fixing the String-Dictionaries

- The json module has another version of load and dump called `json.loads` and `json.dumps`
    - These are designed to process STRINGS instead of files. 
    
- If we use `json.loads` we can convert our string dictionary into an actual dictionary. 

In [8]:
## Use json.loads on the test coordinate
json.loads(test_coord)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

- JSON requires double quotes!

In [9]:
test_coord

"{'latitude': 47.61849, 'longitude': -122.31664}"

In [11]:
## replace single ' with " 
test_coord = test_coord.replace("'", '"')
test_coord

'{"latitude": 47.61849, "longitude": -122.31664}'

In [12]:
## Use json.loads on the test coordinate, again
json.loads(test_coord)

{'latitude': 47.61849, 'longitude': -122.31664}

### Now, how can we apply this same process to the entire column??

In [13]:
## replace ' with " (entire column)
df['coordinates'] = df['coordinates'].str.replace("'", '"')

## apply json.loads
df['coordinates'] = df['coordinates'].apply(json.loads)

In [14]:
## slice out a single test coordinate
test_coord = df.loc[0, 'coordinates']
type(test_coord)

dict

### Using Apply with pd.Series to convert a dictionary column into multiple columns

In [15]:
## use .apply pd.Series to convert a dict to columns
df['coordinates'].apply(pd.Series)

,latitude,longitude
0,47.624577,-122.325577
1,47.618490,-122.316640
2,47.614693,-122.312764
3,47.562070,-122.385090
4,47.597649,-122.313306
...,...,...
850,47.615703,-122.200735
851,47.617996,-122.200456
852,47.664103,-122.313426
853,47.612521,-122.201333


In [16]:
## Concatenate the 2 new columns and drop the original.
df = pd.concat([df, df['coordinates'].apply(pd.Series)], axis = 1)
df = df.drop(columns = 'coordinates')
df.head(2)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,location,phone,display_phone,distance,price,latitude,longitude
0,EwrAoy7XbX8nfemZdDvupA,cornelly-seattle,Cornelly,https://s3-media4.fl.yelpcdn.com/bphoto/HlYYBT...,False,https://www.yelp.com/biz/cornelly-seattle?adju...,101,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"['delivery', 'pickup']","{'address1': '601 Summit Ave E', 'address2': '...",NaN,NaN,2578.701393,NaN,47.624577,-122.325577
1,FVbwpNA1uZEGiM02N4XtUg,blotto-seattle,Blotto,https://s3-media3.fl.yelpcdn.com/bphoto/Q4Y1Cw...,False,https://www.yelp.com/biz/blotto-seattle?adjust...,40,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,[],"{'address1': '1830 12th Ave', 'address2': '', ...",1.206403e+10,(206) 403-1809,1686.862176,NaN,47.618490,-122.316640


## Creating a Simple Map

### Register for MapBox API

Mapbox API: https://www.mapbox.com/

In [ ]:
## Load in mapbox api credentials from .secret
with open('/Users/yawagipong/.secret/mapbox.json') as f:
    login = json.load(f)

login.keys()

- Use the plotly express `set_maptbox_acccess_token` function

In [ ]:
## set mapbox token
px.set_mapbox_access_token(login['api-key'])

In [ ]:
## use scatter_mapbox for M.V.P map
px.scatter_mapbox(df, lat = 'latitude', lon = 'longitude', mapbox_style = 'open-street-map', hover_name = 'name',
                 hover_data = ['price', 'rating', 'location'])

### Adding Hover Data

- We want to show the restaurants:
    - name
    - price range
    - address
    - and if they do delivery or takeout.
    
    
- We can use the `hover_name` and `hover_data` arguments for `px.scatter_mapbox` to add this info!

In [ ]:
## add hover_name (name) and hover_data for price,rating,location


### Fixing the Location Column

In [17]:
## slice out a test address
test_addr = df.loc[0, 'location']
test_addr

"{'address1': '601 Summit Ave E', 'address2': '', 'address3': None, 'city': 'Seattle', 'zip_code': '98102', 'country': 'US', 'state': 'WA', 'display_address': ['601 Summit Ave E', 'Seattle, WA 98102']}"

> Also a string-dictionary...

In [18]:
## replace ' with "
df['location'] = df['location'].str.replace("'", '"')
df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,location,phone,display_phone,distance,price,latitude,longitude
0,EwrAoy7XbX8nfemZdDvupA,cornelly-seattle,Cornelly,https://s3-media4.fl.yelpcdn.com/bphoto/HlYYBT...,False,https://www.yelp.com/biz/cornelly-seattle?adju...,101,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"['delivery', 'pickup']","{""address1"": ""601 Summit Ave E"", ""address2"": ""...",NaN,NaN,2578.701393,NaN,47.624577,-122.325577
1,FVbwpNA1uZEGiM02N4XtUg,blotto-seattle,Blotto,https://s3-media3.fl.yelpcdn.com/bphoto/Q4Y1Cw...,False,https://www.yelp.com/biz/blotto-seattle?adjust...,40,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,[],"{""address1"": ""1830 12th Ave"", ""address2"": """", ...",1.206403e+10,(206) 403-1809,1686.862176,NaN,47.618490,-122.316640
2,fxyHWmfzcdjImgQ_UYHoTw,bar-cotto-seattle,Bar Cotto,https://s3-media3.fl.yelpcdn.com/bphoto/07DoPF...,False,https://www.yelp.com/biz/bar-cotto-seattle?adj...,343,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"['delivery', 'pickup']","{""address1"": ""1546 15th Ave"", ""address2"": """", ...",1.206839e+10,(206) 838-8081,1292.526796,$$,47.614693,-122.312764
3,IrohtoYjnAR_vc6w6CRCxA,moto-seattle,Moto,https://s3-media2.fl.yelpcdn.com/bphoto/FBmvjy...,False,https://www.yelp.com/biz/moto-seattle?adjust_c...,143,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,['delivery'],"{""address1"": ""4526 42nd Ave SW"", ""address2"": ""...",1.206421e+10,(206) 420-8880,8784.075722,$$,47.562070,-122.385090
4,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media4.fl.yelpcdn.com/bphoto/A08GOZ...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,362,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],"{""address1"": ""525 Rainier Ave S"", ""address2"": ...",1.206330e+10,(206) 329-5133,2166.834011,$$,47.597649,-122.313306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
850,rcDbUcs83gvoRxhaTmFfnQ,cypress-lounge-and-wine-bar-bellevue,Cypress Lounge & Wine Bar,https://s3-media2.fl.yelpcdn.com/bphoto/DsFhIb...,False,https://www.yelp.com/biz/cypress-lounge-and-wi...,68,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",3.5,[],"{""address1"": ""600 Bellevue Way NE"", ""address2""...",1.425638e+10,(425) 638-1000,7123.208902,$$,47.615703,-122.200735
851,H_gsS9J920u_hK7jekobqw,suite-bellevue-4,Suite,https://s3-media1.fl.yelpcdn.com/bphoto/jdYMfv...,False,https://www.yelp.com/biz/suite-bellevue-4?adju...,232,"[{'alias': 'lounges', 'title': 'Lounges'}, {'a...",3.0,['delivery'],"{""address1"": ""10500 NE 8th St"", ""address2"": ""S...",1.425680e+10,(425) 679-6951,7158.825860,$$,47.617996,-122.200456
852,AsgtzyqpdjuPyjaEUVcUXA,jewel-of-india-seattle-3,Jewel Of India,https://s3-media2.fl.yelpcdn.com/bphoto/qJGMnl...,False,https://www.yelp.com/biz/jewel-of-india-seattl...,411,"[{'alias': 'indpak', 'title': 'Indian'}]",3.5,"['pickup', 'delivery']","{""address1"": ""4735 University Way NE"", ""addres...",1.206524e+10,(206) 523-5275,5825.461180,$$,47.664103,-122.313426
853,oniQ68t6VR1naVtUXmfETQ,pokeworks-bellevue-2,Pokeworks,https://s3-media1.fl.yelpcdn.com/bphoto/DqSOK8...,False,https://www.yelp.com/biz/pokeworks-bellevue-2?...,284,"[{'alias': 'poke', 'title': 'Poke'}, {'alias':...",4.0,"['pickup', 'delivery']","{""address1"": ""222 Bellevue Way NE"", ""address2""...",1.425214e+10,(425) 214-1182,7072.982860,$$,47.612521,-122.201333


In [19]:
## apply json.loads
df['location'] = df['location'].apply(json.loads)
df

JSONDecodeError: Expecting value: line 1 column 62 (char 61)

> Ruh roh....

- Hmm, let's slice out a test_address again and let's write a function to accomplish this instead.
    - We can use try and except in our function to get around the errors.

### Fixing Addresses - with a custom function


In [20]:
## slice out test address 
test_addr = df.loc[0, 'location']
test_addr

'{"address1": "601 Summit Ave E", "address2": "", "address3": None, "city": "Seattle", "zip_code": "98102", "country": "US", "state": "WA", "display_address": ["601 Summit Ave E", "Seattle, WA 98102"]}'

In [21]:
## write a function to just run json.loads on the address
def fix_address(test_addr):
    try:
        return json.loads(test_addr)
    except:
        return 'Error'

In [22]:
## test applying our function
df['location'].apply(fix_address)

0                                                  Error
1                                                  Error
2      {'address1': '1546 15th Ave', 'address2': '', ...
3                                                  Error
4      {'address1': '525 Rainier Ave S', 'address2': ...
                             ...                        
850    {'address1': '600 Bellevue Way NE', 'address2'...
851    {'address1': '10500 NE 8th St', 'address2': 'S...
852                                                Error
853                                                Error
854                                                Error
Name: location, Length: 855, dtype: object

- It worked! Now let's save this as a new column (display_location),
and then let's investigate the businesses that had an "ERROR".

In [23]:
### save a new display_location column using our function
df['display_location'] = df['location'].apply(fix_address)

In [27]:
## filter for businesses with display_location == "ERROR"
errors = df[df['display_location'] == 'Error']
errors

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,location,phone,display_phone,distance,price,latitude,longitude,display_location
0,EwrAoy7XbX8nfemZdDvupA,cornelly-seattle,Cornelly,https://s3-media4.fl.yelpcdn.com/bphoto/HlYYBT...,False,https://www.yelp.com/biz/cornelly-seattle?adju...,101,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"['delivery', 'pickup']","{""address1"": ""601 Summit Ave E"", ""address2"": ""...",NaN,NaN,2578.701393,NaN,47.624577,-122.325577,Error
1,FVbwpNA1uZEGiM02N4XtUg,blotto-seattle,Blotto,https://s3-media3.fl.yelpcdn.com/bphoto/Q4Y1Cw...,False,https://www.yelp.com/biz/blotto-seattle?adjust...,40,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,[],"{""address1"": ""1830 12th Ave"", ""address2"": """", ...",1.206403e+10,(206) 403-1809,1686.862176,NaN,47.618490,-122.316640,Error
3,IrohtoYjnAR_vc6w6CRCxA,moto-seattle,Moto,https://s3-media2.fl.yelpcdn.com/bphoto/FBmvjy...,False,https://www.yelp.com/biz/moto-seattle?adjust_c...,143,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,['delivery'],"{""address1"": ""4526 42nd Ave SW"", ""address2"": ""...",1.206421e+10,(206) 420-8880,8784.075722,$$,47.562070,-122.385090,Error
9,wi-mbKEz4Ha1YYaOB8zHnw,kōbo-pizza-seattle,Kōbo Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/6et4T9...,False,https://www.yelp.com/biz/k%C5%8Dbo-pizza-seatt...,32,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,[],"{""address1"": ""714 E Pike St"", ""address2"": None...",1.253693e+10,(253) 693-0207,2025.139396,NaN,47.614400,-122.322710,Error
11,ugTsEtjvwRhteac_6JcuPw,italian-family-pizza-seattle,Italian Family Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/SgiSrJ...,False,https://www.yelp.com/biz/italian-family-pizza-...,1004,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"['delivery', 'pickup']","{""address1"": ""1028 Madison St"", ""address2"": No...",1.206538e+10,(206) 538-0040,2271.307105,$$,47.609370,-122.325460,Error
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
833,3cLNDJ2vI29LWNvU7vVIuQ,capital-one-café-bellevue-2,Capital One Café,https://s3-media1.fl.yelpcdn.com/bphoto/gcL_qI...,False,https://www.yelp.com/biz/capital-one-caf%C3%A9...,42,"[{'alias': 'banks', 'title': 'Banks & Credit U...",4.0,[],"{""address1"": ""400 Bellevue Way NE"", ""address2""...",1.425372e+10,(425) 372-0250,7071.129218,$,47.614080,-122.201363,Error
834,Nihv2RfuTZ4TI8BQl3anmA,qfc-kirkland-4,QFC,https://s3-media2.fl.yelpcdn.com/bphoto/NYz4rH...,False,https://www.yelp.com/biz/qfc-kirkland-4?adjust...,32,"[{'alias': 'grocery', 'title': 'Grocery'}]",3.0,[],"{""address1"": ""457 Central Way"", ""address2"": """"...",1.425827e+10,(425) 827-2205,10212.263314,$$,47.678048,-122.199308,Error
852,AsgtzyqpdjuPyjaEUVcUXA,jewel-of-india-seattle-3,Jewel Of India,https://s3-media2.fl.yelpcdn.com/bphoto/qJGMnl...,False,https://www.yelp.com/biz/jewel-of-india-seattl...,411,"[{'alias': 'indpak', 'title': 'Indian'}]",3.5,"['pickup', 'delivery']","{""address1"": ""4735 University Way NE"", ""addres...",1.206524e+10,(206) 523-5275,5825.461180,$$,47.664103,-122.313426,Error
853,oniQ68t6VR1naVtUXmfETQ,pokeworks-bellevue-2,Pokeworks,https://s3-media1.fl.yelpcdn.com/bphoto/DqSOK8...,False,https://www.yelp.com/biz/pokeworks-bellevue-2?...,284,"[{'alias': 'poke', 'title': 'Poke'}, {'alias':...",4.0,"['pickup', 'delivery']","{""address1"": ""222 Bellevue Way NE"", ""address2""...",1.425214e+10,(425) 214-1182,7072.982860,$$,47.612521,-122.201333,Error


In [30]:
## slice out a new test address and inspect, note 437 has an error
test_addr = df.loc[437, 'location']
test_addr

'{"address1": "1001 Broadway", "address2": "", "address3": None, "city": "Seattle", "zip_code": "98122", "country": "US", "state": "WA", "display_address": ["1001 Broadway", "Seattle, WA 98122"]}'

> After some more investigation, we would find a few issues with these "ERROR" rows.
1. They contained None.
2. They contained an apostrophe in the name.
3. ...?

### Possible Fixes (if we care to/have the time)


- Use Regular Expressions to find an fix the display addresses with "'" in them
- Use string split to split on the word display address.
    - Then use string methods to clean up

### Moving Forward without those rows (for now)

In [32]:
## remove any rows where display_location == 'ERROR'
df = df.loc[df['display_location']!= 'ERROR']

- We want the "display_address" key from the "display_location" dictionaries.
- We could use a .apply and a lamda to slice out the desired key.

In [ ]:
## use apply and lambda to slice correct key
df['display_address'] = df['display_location'].apply(lambda x :x['display_address'])
df.head(3)

- Almost done! We want to convert display_address to a string instead a list of strings.
- We can use the string method .join to do so!

In [ ]:
## slice out a test_address
test_add = df.loc[339,'display_address']
test_add

In [ ]:
## test using .join with a "\n"
'\n'.join(test_add)

In [ ]:
## apply the join to every row with a lambda
df['Address'] = df['display_address'].apply(lambda x: '\n'.join(x))
df

### Final Map

In [ ]:
## make ourn final map and save as varaible
pfig = px.scatter_mapbox(df, lat='latitude',lon='longitude',
                  mapbox_style='open-street-map', hover_name='name',
                         hover_data=['price','rating','Address'],
                 color='rating',
                        title='Restaurants with Pizza in Seattle, WA')


pfig.show()

#### HTML Uses `<br>` instead of `\n`

In [ ]:
## remake the final address column with <br> instead 

## plot the final map
pfig.write_html('final_map.html')

In [ ]:
## use fig.write_html to save map
